<a href="https://colab.research.google.com/github/itrnltw/rDownloader/blob/main/Recurbate_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurbate Downloader


## Mount Drive

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
from google.colab import drive
drive.mount('/content/gdrive')

## Run Download

In [ ]:
#@markdown <br><center><img src='https://recurbate.com/favicon.png' height="50" alt="Recurbate-logo"/></center>
#@markdown <center><h3Download Video</h3></center><br>
import requests, random, os
from tqdm import tqdm
from bs4 import BeautifulSoup
os.system('')

kepala = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

# with open(r'D:\py\working_proxies.txt') as f:
#     proxies = f.readlines()
proxies = []
# f.close()

def ambilProxy(proxies):
    if len(proxies) > 0:
        proxy = random.choice(proxies)
        return proxy
    else:
        print("Proxies Empty")
        quit()
        return quit()

def ambilID(video_url, proxy):
    try:
        ambil_data = requests.get(video_url, headers=kepala, proxies={'http':proxy,'https':proxy}, timeout=5)
        soup = BeautifulSoup(ambil_data.text, 'html.parser')
        tombol = soup.find('button')
        if tombol:
            data_video_id = tombol.attrs.get('data-video-id')
            data_token = tombol.attrs.get('data-token')
            print(f"[+] \033[1;32;40m{data_token} {data_video_id}\33[0m")
            return data_video_id, data_token
        else:
            print(f"[-] \033[1;31;40mFailed to find ID & Token: {soup.text}\33[0m")
            return None, None
    except (requests.exceptions.RequestException, ValueError):
        print(f"[-] \033[1;31;40mFailed to access ID & Token\33[0m")
        return None, None

def ambileFileDariAPI(data_video_id, data_token, proxy):
    try:
        urlAPI = "https://recurbate.com/api/get.php?video="+data_video_id+"&token="+data_token
        ambil_video = requests.get(urlAPI, headers=kepala, proxies={'http':proxy,'https':proxy}, timeout=5)
        soup = BeautifulSoup(ambil_video.text, 'html.parser')
        sumber = soup.find('source')
        if sumber:
            hasilnya = sumber.attrs.get('src')
            print(f"\033[1;32;40m{hasilnya}\33[0m")
            return hasilnya
        else:
            print(f"[-] \033[1;31;40mFailed to find video source: {soup.text}\33[0m")
            return None
    except (requests.exceptions.RequestException, ValueError):
        print(f"[-] \033[1;31;40mFailed to access URL File\33[0m")
        return None

def download_video(url, file_name, proxy):
    try:
        response = requests.get(url, headers=kepala, stream=True, proxies={'http':proxy,'https':proxy}, timeout=5)
        total_size = int(response.headers.get("Content-Length", 0))
        block_size = 1024
        t=tqdm(total=total_size, unit='B', unit_scale=True)
        with open(file_name, "wb") as f:
            for data in response.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
        if total_size != 0 and t.n != total_size:
            print("\033[1;31;40mERROR, something went wrong\33[0m")
            return None
        elif response.status_code == 410:
            print(response.status_code)
            return None
        else:
            return f"\033[1;32;40mFile {file_name.split('/')[5]} Downloaded\33[0m"
    except (requests.exceptions.RequestException, ValueError):
        print(f"[-] \033[1;31;40mFailed to Download\33[0m")
        return None

def main():
    video_url = "https://recurbate.com/play.php?video=44290255" #@param {type:"string"}
    path = "/content/gdrive/Shareddrives/#xTemp" #@param {type:"string"}
    id, token = None, None
    alamat_File = None
    downloaded = None
    
    withProxyOrNot = True #@param {type:"boolean"}
    if not withProxyOrNot:
        id, token = ambilID(video_url, None)
        if not id and not token:
            return quit()
        alamat_File = ambileFileDariAPI(id, token, None)
        if not alamat_File:
            return quit()
        file_name = rf"E:\Downloaded\{alamat_File.split('?')[0].split('/')[4]}"
        download_video(alamat_File, file_name, None)
        if not downloaded:
            return quit()
    else:
        inputproxy = "31.186.239.245:8080 116.0.61.122:3128" #@param {type:"string"}
        proxies = inputproxy.split(' ')
        if inputproxy == "":
            print("Input Proxy First")
            quit()
        else:
            counter = 1
            proxy = ambilProxy(proxies)
            print(f"[{counter}] Working with {proxy}")
            while not downloaded :
                while not alamat_File :
                        while not id and not token :
                            id, token = ambilID(video_url, proxy)
                            if not id and not token:
                                proxies.remove(proxy)
                                proxy = ambilProxy(proxies)
                                counter += 1
                                print(f"[{counter}] Changing Proxy to {proxy}")
                        alamat_File = ambileFileDariAPI(id, token, proxy)
                        if not alamat_File:
                            id, token = None, None
                            proxies.remove(proxy)
                            proxy = ambilProxy(proxies)
                            counter += 1
                            print(f"[{counter}] Changing Proxy to {proxy}")
                        elif len(proxies) == 0:
                            quit()
                        else:
                            file_name = rf"{path+'/'+alamat_File.split('?')[0].split('/')[4]}"
                            # file_name = rf"E:\Downloaded\{alamat_File.split('?')[0].split('/')[4]}"
                download_video(alamat_File, file_name, proxy)
                if not downloaded:
                    id, token = None, None
                    alamat_File = None
                    proxies.remove(proxy)
                    proxy = ambilProxy()
                    counter += 1
                    print(f"[{counter}] Changing Proxy to {proxy}")
                    

if __name__=='__main__':
    main()